Farmer problem (deterministic policy)

Mauricio Tombini, Jordan Jalving, and Victor M. Zavala 

University of Wisconsin-Madison, 2016

In [1]:
using JuMP 
using Ipopt 

In [2]:
# Model parameters

NS = 5;                  # number of scenarios
S = collect(1:NS);       # scenario set
P = collect(1:3);        # set of crops (1=wheat,2=corn,3=beets)

# Data 
prcost = zeros(3)          # production (planting) cost
prcost[1] = 150;
prcost[2] = 230;
prcost[3] = 260;

pcost = zeros(3)           # purchase cost
pcost[1] = 238;
pcost[2] = 210;
pcost[3] = 0;

scost = zeros(3)           # sales cost
scost[1] = 170;
scost[2] = 150;
scost[3] = 36;

demand = zeros(3)          # demand
demand[1] = 200;
demand[2] = 240;
demand[3] = 0;

# assign random data (first set all equal to mean forecast)
yield = zeros(length(S),3) # yields
yield[S,1] = 2.5;
yield[S,2] = 3.0;
yield[S,3] = 20;

## Solve deterministic problem using average forecast for yields

In [3]:
# Model 
mp = Model(solver=IpoptSolver(print_level=0))

@variable(mp, x[S,P] >= 0)    # acres devoted to crops
@variable(mp, y[S,P] >= 0)    # crops purchase
@variable(mp, w[S,P] >= 0)    # crops sold
@expression(mp, Cost[s in S], sum(prcost[j]*x[s,j] + pcost[j]*y[s,j] - scost[j]*w[s,j] for j in P))
@variable(mp, cost[s in S])

@constraint(mp, varcost[s in S], cost[s] == Cost[s]) 
@constraint(mp, cap[s in S], sum(x[s,j] for j in P) <= 500)
@constraint(mp, bal[s in S,j in P], yield[s,j]*x[s,j]+y[s,j]-w[s,j] >= demand[j]) 
@constraint(mp, sellb[s in S], w[s,3] <= 6000)
@constraint(mp, buyb[s in S], y[s,3] <= 0)
@constraint(mp, nonant[s in S,j in P], x[1,j] == x[s,j])

@objective(mp, Min, (1/NS)*sum(cost[s] for s in S))

solve(mp)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************



:Optimal

## Now solve recourse problem using deterministic first-stage solution and actual yields

In [4]:
xs = getvalue(x); 

In [5]:
# set actual random data
yield[1,3] = 10;
yield[2,3] = 15;
yield[3,3] = 20;
yield[4,3] = 25;
yield[5,3] = 30;

# Model 
m = Model(solver=IpoptSolver(print_level=0))

@variable(m, y[S,P] >= 0)    # crops purchase
@variable(m, w[S,P] >= 0)    # crops sold
@variable(m, cost[s in S])

@expression(m, Cost[s in S], sum(prcost[j]*xs[s,j] + pcost[j]*y[s,j] - scost[j]*w[s,j] for j in P))
@constraint(m, varcost[s in S], cost[s] == Cost[s]) 
@constraint(m, bal[s in S,j in P], yield[s,j]*xs[s,j]+y[s,j]-w[s,j] >= demand[j]) 
@constraint(m, sellb[s in S], w[s,3] <= 6000)
@constraint(m, buyb[s in S], y[s,3] <= 0)

@objective(m, Min, (1/NS)*sum(cost[s] for s in S))

solve(m)

In [6]:
# Results
println("x = ",getvalue(x))
println("")
println("y = ",getvalue(y))
println("")
println("w = ",getvalue(w))
println("")
println("cost = ",getvalue(cost))
println("")
println("obj: ", getobjectivevalue(m))

x = x: 2 dimensions:
[1,:]
  [1,1] = 120.00000279984634
  [1,2] = 79.99999920015239
  [1,3] = 300.00000299995565
[2,:]
  [2,1] = 120.00000279984634
  [2,2] = 79.99999920015239
  [2,3] = 300.00000299995565
[3,:]
  [3,1] = 120.00000279984634
  [3,2] = 79.99999920015239
  [3,3] = 300.00000299995565
[4,:]
  [4,1] = 120.00000279984634
  [4,2] = 79.99999920015239
  [4,3] = 300.00000299995565
[5,:]
  [5,1] = 120.00000279984634
  [5,2] = 79.99999920015239
  [5,3] = 300.00000299995565

y = y: 2 dimensions:
[1,:]
  [1,1] = 0.0
  [1,2] = 2.3798087768711324e-6
  [1,3] = 9.65532035803641e-9
[2,:]
  [2,1] = 0.0
  [2,2] = 2.3798087768711324e-6
  [2,3] = 9.646982445737395e-9
[3,:]
  [3,1] = 0.0
  [3,2] = 2.3798087768711324e-6
  [3,3] = 6.278599013474232e-13
[4,:]
  [4,1] = 0.0
  [4,2] = 2.3798087768711324e-6
  [4,3] = 3.280541440718195e-20
[5,:]
  [5,1] = 0.0
  [5,2] = 2.3798087768711324e-6
  [5,3] = 1.608959349234551e-20

w = w: 2 dimensions:
[1,:]
  [1,1] = 100.00000798972646
  [1,2] = 0.0
  [1,3] =